<h1>Proyecto ETL</h1>

<h2>Presentación</h2>

<p>La idea de este proyecto es realizar un proceso basico de extraer, transformar y cargar para practicar mi forma de documentar y usar github así como demostrar conocimiento basico de request, pandas y SQL. </p>

In [1]:
import requests
import random
import pandas as pd
from sqlalchemy import create_engine

<p>Encontre esta api que me parecio interesante y facil de usar asi que voy a investigarla usando postman y la documentacion</p>

http://universities.hipolabs.com/
https://github.com/Hipo/university-domains-list-api

<p>Me aseguro de que funcione en jupyter</p>

In [2]:
url_api = "http://universities.hipolabs.com/search"


respuesta = requests.get(url_api)

#compruebo de que funcione, caso contrario que imprima el codigo de error.
if respuesta.status_code == 200:
    universidades = respuesta.json()
    #A modo de ejemplo elijo una universidad al azar y la imprimo para asegurarme de que funcione
    universidadejemplo = random.choice(universidades)
    print(universidadejemplo)
else:
    print('La solicitud no fue exitosa. Código de estado:', respuesta.status_code)

{'state-province': None, 'country': 'Tanzania, United Republic of', 'domains': ['stjosephtanzania.com'], 'web_pages': ['http://www.stjosephtanzania.com/sjcet/index.html'], 'alpha_two_code': 'TZ', 'name': 'St. Joseph Group of Institutes'}


<p>La api devuelve una lista de universidades en .json, para reducir la lista voy pegarle al endpoint que devuelve solo las de Argentina</p>

http://universities.hipolabs.com/search?country=Argentina

<h2>Extraer</h2>

<p>Voy a definir una funcion extrer que va a guardar los datos en formato .json</p>

In [3]:
def extraer():
    """ Esta api extrae datos desde
    http://universities.hipolabs.com/
    """
    url = 'http://universities.hipolabs.com/search?country=Argentina'
    datos = requests.get(url).json()
    return datos

<h2>Transformar</h2>

<p>Voy a realizar una transformacion antes de guardar los datos, usando pandas</p>

In [4]:
df = pd.DataFrame(extraer())
df

,state-province,country,domains,web_pages,alpha_two_code,name
0,Buenos Aires,Argentina,[atlantida.edu.ar],[http://www.atlantida.edu.ar/],AR,Universidad Atlantida Argentina
1,Buenos Aires,Argentina,[austral.edu.ar],[http://www.austral.edu.ar/],AR,Universidad Austral Buenos Aires
2,Ciudad Autónoma de Buenos Aires,Argentina,[caece.edu.ar],[http://www.caece.edu.ar/],AR,"Universidad CAECE, Buenos Aires"
3,Ciudad Autónoma de Buenos Aires,Argentina,[cema.edu.ar],[http://www.cema.edu.ar/],AR,Instituto Universitario CEMA
4,Ciudad Autónoma de Buenos Aires,Argentina,[iese.edu.ar],[http://www.iese.edu.ar/],AR,Instituto de Enseñanza Superior del Ejército
...,...,...,...,...,...,...
81,Buenos Aires,Argentina,[untref.edu.ar],[http://www.untref.edu.ar/],AR,Universidad Nacional de Tres de Febrero
82,Ciudad Autónoma de Buenos Aires,Argentina,[utdt.edu],[http://www.utdt.edu/],AR,Universidad Torcuato di Tella
83,Ciudad Autónoma de Buenos Aires,Argentina,[utn.edu.ar],[http://www.utn.edu.ar/],AR,Universidad Tecnológica Nacional
84,Ciudad Autónoma de Buenos Aires,Argentina,[vaneduc.edu.ar],[http://www.vaneduc.edu.ar/uai/],AR,Universidad Abierta Interamericana


In [5]:
df.describe()

,state-province,country,domains,web_pages,alpha_two_code,name
count,65,86,86,86,86,86
unique,17,1,86,86,1,86
top,Buenos Aires,Argentina,[atlantida.edu.ar],[http://www.atlantida.edu.ar/],AR,Universidad Atlantida Argentina
freq,20,86,1,1,86,1


<p>Verifico parte de la estructura del dataframe, no tiene inconvenientes respecto a la "limpieza" de los datos, solamente queda filtrarlos, tener en cuenta que ciudad y provincia de buenos aires aparecen separados</p>

In [6]:
df = df.drop(columns=['alpha_two_code','country'])

<p>Ya que por medio de la api filtre a las universidades argentinas, resultan redundantes ambas columnas</p>

In [7]:
df = df.loc[(df["state-province"] == "Buenos Aires")| (df["state-province"] == "Ciudad Autónoma de Buenos Aires")]
df.head()

,state-province,domains,web_pages,name
0,Buenos Aires,[atlantida.edu.ar],[http://www.atlantida.edu.ar/],Universidad Atlantida Argentina
1,Buenos Aires,[austral.edu.ar],[http://www.austral.edu.ar/],Universidad Austral Buenos Aires
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],[http://www.caece.edu.ar/],"Universidad CAECE, Buenos Aires"
3,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],[http://www.cema.edu.ar/],Instituto Universitario CEMA
4,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],[http://www.iese.edu.ar/],Instituto de Enseñanza Superior del Ejército


In [8]:
df = df.reset_index(drop=True)

<p>Ahora lo voy a cargar a una base de datos relacional de forma local con SqlAlchemy</p>

In [9]:
def cargar(df:pd.DataFrame):
    """ carga los datos en una base de datos sqllite """
    disk_engine = create_engine('sqlite:///bdatos.db')
    df.to_sql('cal_uni', disk_engine, if_exists='replace')

In [10]:
cargar(df)

ProgrammingError: (sqlite3.ProgrammingError) Error binding parameter 3: type 'list' is not supported
[SQL: INSERT INTO cal_uni ("index", "state-province", domains, web_pages, name) VALUES (?, ?, ?, ?, ?)]
[parameters: [(0, 'Buenos Aires', ['atlantida.edu.ar'], ['http://www.atlantida.edu.ar/'], 'Universidad Atlantida Argentina'), (1, 'Buenos Aires', ['austral.edu.ar'], ['http://www.austral.edu.ar/'], 'Universidad Austral Buenos Aires'), (2, 'Ciudad Autónoma de Buenos Aires', ['caece.edu.ar'], ['http://www.caece.edu.ar/'], 'Universidad CAECE, Buenos Aires'), (3, 'Ciudad Autónoma de Buenos Aires', ['cema.edu.ar'], ['http://www.cema.edu.ar/'], 'Instituto Universitario CEMA'), (4, 'Ciudad Autónoma de Buenos Aires', ['iese.edu.ar'], ['http://www.iese.edu.ar/'], 'Instituto de Enseñanza Superior del Ejército'), (5, 'Buenos Aires', ['inun.edu.ar'], ['http://www.inun.edu.ar/'], 'Instituto Universitario de Estudios Navales y Marítimos'), (6, 'Ciudad Autónoma de Buenos Aires', ['itba.edu.ar'], ['h